In [4]:
import random
import gc
from scipy.special import comb
from scipy.stats import entropy
from math import log, factorial, isfinite

from entropy_stuff import *
from Convolution_stuff import *
from Epsilon_choice import low_eps, A
from Multinomial import multinomial
from Truelog import truelog, flog

from Samplerz import sample_gaussian

#from IPython.display import clear_output
#from multiprocessing import Pool

In [8]:
quentropy([1/27,2/27,2/27,2/27,4/27,4/27,4/27,8/27],1),3*quentropy([1/3,2/3],1),3*truelog(3)-54/27

(2.7548875021634687, 2.7548875021634682, 2.754887502163469)

In [1]:
float(1.17*sqrt(12289/2048))

2.8660196105754623

In [8]:
(1071.949877-499*quentropy([1/16,1/4,3/8,1/4,1/16],1))/499

0.11755708406301843

In [2]:
float(1.17*sqrt(12289/1024))

4.053163803303075

In [24]:
quentropy([1/16,3/16,5/16,7/16],1)

1.7489992230622806

In [135]:
sum_of_roots={}
    
def true_sum_of_roots(a, b):
    s=0
    for i in range(a, b+1):
        s+=i**(1/2)
    return (s)

for n in range(100):
    sum_of_roots[n]=true_sum_of_roots(0, n)

def integral_approx(a,b):
    return (int(2/3*(b+1)**(3/2))-int(2/3*(a)**(3/2))+int(2/3*(b)**(3/2))-int(2/3*(a-1)**(3/2)))/2

def approx_sum_of_roots(a,b):
    if b-a<200:
        return true_sum_of_roots(a, b)
    else:
        return integral_approx(a,b)

In [138]:
class parlists:
    def __init__(self, const, dic, num, p):
        self.c = const
        self.dic = dic
        self.len = len(dic)
        self.count = num
        self.p = p
    
    def __repr__(self):
        if self.c<0:
            return("List with " + str(self.len) + " clusters, with probability at least 2^(-H(.)n+" + str(-self.c) + "), total probability is " + str(float(self.p)))
        else:
            return("List with " + str(self.len) + " clusters, with probability at least 2^(-H(.)n-" + str(self.c) + "), total probability is " + str(float(self.p)))
        
empty_list=parlists(-float('inf'),[],0,0)

class distributions:
    def __init__(self, dist):
        cumu=0
        for key in dist:
            cumu+=dist[key]
        if cumu!=1:
            print("Not a probability distribution")
        else:
            probabilities, multiplicities=multiplicity(dist)
            eta=len(probabilities)-1
            self.eta = eta
            self.d = dist
            self.dist = dist
            self.p = probabilities
            self.log_p = []
            for prob in self.p:
                self.log_p += [-truelog(prob,2)]
            self.m = multiplicities
            self.entropy = dist_entropy(dist, 1)
            self.kcl={}
            self.keys=[]
    
    def prob(self, i):
        return self.d.get(i,0)
    
    def vec_prob(self, L):
        p=0
        if len(L)-1<=self.eta:
            p=1
            for i in range(len(L)):
                p*=self.prob(i)**L[i]
        return p
    
    def kc(self, n):
        return self.kcl.get(n,empty_list)
    
    def create_parlist(self, n, p_limit,cur_j,high_n,L,p, if_print=True):
        if cur_j==self.eta and self.log_p[self.eta]*high_n<=p_limit:
            L[cur_j]=high_n
            if if_print:
#                clear_output(wait=True)
                print(L)
            count=count_L(L, self.m)
            self.kcl[n].dic+=[[L.copy(),p*self.p[cur_j]**high_n,count]]
            self.kcl[n].count+=count
            self.kcl[n].p+=count*p*self.p[cur_j]**high_n
            L[cur_j]='-'
        elif high_n==0:
            for j in range(cur_j, self.eta+1):
                L[j]=0
            if if_print:
#                clear_output(wait=True)
                print(L)
            count=count_L(L, self.m)
            self.kcl[n].dic+=[[L.copy(),p*self.p[cur_j]**high_n,count]]
            self.kcl[n].count+=count
            self.kcl[n].p+=count*p*self.p[cur_j]**high_n
            for j in range(cur_j, self.eta+1):
                L[j]='-'
        elif cur_j<self.eta:
            if p_limit//self.log_p[cur_j]>=high_n:
    #            print(global_log_p[cur_j])
    #            print(floor(p_limit/global_log_p[cur_j])+1)
                for i in range(high_n+1):
    #                if cur_j==0:
    #                    print(i)
                    L[cur_j]=i
                    self.create_parlist(n, p_limit-i*self.log_p[cur_j],cur_j+1,high_n-i,L,p*self.p[cur_j]**i, if_print=if_print)
    #                L[cur_j]='-'
    
    def sam_parlist(self, n, c=0, if_ret=True, if_print=False):
        if self.kc(n).c<c:
            if self.eta==0:
                self.kcl[n]=parlists(c,[[n]])
            else:
                gc.collect()
                self.kcl[n]=parlists(-float('inf'),[],0,0)
                self.create_parlist(n,self.entropy*n+c,0,n,(self.eta+1)*['-'],1, if_print=if_print)
                self.kcl[n].dic.sort(key=lambda x: x[1], reverse=True)
                self.kcl[n].len=len(self.kcl[n].dic)
                self.kcl[n].c=c
                gc.collect()
                out=self.kc(n)
        elif self.kc(n).c>c:
            out_L=[]
            p=0
            count=0
            for l in self.kc(n).dic:
                if truelog(l[1])<-self.entropy*n-c:
                    break
                out_L+=[l]
                p+=l[1]*l[2]
                count+=l[2]
            out=parlists(c, out_L, count, p)
        else:
            out=self.kc(n)
        if if_ret:
            return out
        
    def sam_list(self, n, if_ret=True, if_print=False):
        c=-n*(self.entropy-max(self.log_p))+1
        return self.sam_parlist(n, c=c, if_ret=if_ret, if_print=if_print)
        
    def count_til_p(self,n,c=0,ret_l=True, if_print=True):
        if self.kc(n).c<c:
            self.sam_parlist(n,c,if_print=if_print)
            out=self.kc(n).count
        elif self.kc(n).c==c:
            out=self.kc(n).count
        else:
            out=0
            i=0
            L=self.kc(n).dic
            while i<len(L) and truelog(L[i][1])>-self.entropy*n-c:
                out+=L[i][2]
                i+=1
        if ret_l:
            return truelog(out)
        return out
    
    def raw_data(self, low_n, high_n, c=0, delete_after=False, step=1, aborts=True):
        if aborts:
            self.make_tablepoints(low_n, high_n, c=c, delete_after=delete_after, step=step)
        else:
            self.make_tablepoints_no_aborts(low_n, high_n, c=c, delete_after=delete_after, step=step)
    
    def make_tablepoints(self, low_n, high_n, c=0, delete_after=False, step=1):
        print("n p coresize Eclassic Equantum")
        for n in range(low_n, high_n+1, step):
            L=self.sam_parlist(n, c=c, if_ret=True, if_print=False)
            p=0
            count=0
            ELZ=0
            Equantum=0
            for l in L.dic:
                p+=l[1]*l[2]
                ELZ+=l[1]*l[2]*(count+(l[2]+1)/2)
                Equantum+=l[1]*approx_sum_of_roots(count+1,count+l[2])
                count+=l[2]
            ELZ+=(1-p)*count
            print(n,format(float(p),'f'),truelog(count),truelog(ELZ),truelog(Equantum))
            if delete_after:
                del(self.kcl[n])
                gc.collect()
    
    def make_tablepoints_no_aborts(self, low_n, high_n, c=0, delete_after=False, step=1):
        print("n p coresize Eclassic Enoabort Equantum")
        for n in range(low_n, high_n+1, step):
            L=self.sam_list(n, if_ret=True, if_print=False)
            p1=0
            p2=0
            count1=0
            count2=0
            ELZ1=0
            ELZ2=0
            Equantum=0
            for l in L.dic:
                if truelog(l[1])>=-self.entropy*n-c:
                    p1+=l[1]*l[2]
                    ELZ1+=l[1]*l[2]*(count1+(l[2]+1)/2)
                    Equantum+=l[1]*approx_sum_of_roots(count1+1,count1+l[2])
                    count1+=l[2]
#                else:
#                    ELZ1+=(1-p1)*l[2]
                p2+=l[1]*l[2]
                ELZ2+=l[1]*l[2]*(count2+(l[2]+1)/2)
                count2+=l[2]
            ELZ1+=(1-p1)*count1
            print(n,format(float(p1),'f'),truelog(count1),truelog(ELZ1),truelog(ELZ2),truelog(Equantum))
            if delete_after:
                del(self.kcl[n])
                gc.collect()
            
            
            
            
    
        
def count_L(L,m):
    out=1
    for i in range(len(L)):
        out*=m[i]**L[i]
    return out*multinomial(L)



CBD_list={0:distributions({0:1}),1:distributions({-1:1/4,0:1/2,1:1/4})}

def CBD(eta):
    global CBD_list
    CBD_list[eta]=CBD_list.get(eta,distributions(self_convolution(CBD_list[1].d,eta)))
    return CBD_list[eta]


falcon512dist={-20: 1/1048576,
 -19: 1/524288,
 -18: 3/1048576,
 -17: 19/1048576,
 -16: 43/1048576,
 -15: 117/1048576,
 -14: 65/262144,
 -13: 19/32768,
 -12: 1323/1048576,
 -11: 2613/1048576,
 -10: 4887/1048576,
 -9: 4359/524288,
 -8: 7321/524288,
 -7: 23297/1048576,
 -6: 34803/1048576,
 -5: 187/4096,
 -4: 63141/1048576,
 -3: 609/8192,
 -2: 45531/524288,
 -1: 25219/262144,
 0: 52049/524288,
 1: 25219/262144,
 2: 45531/524288,
 3: 609/8192,
 4: 63141/1048576,
 5: 187/4096,
 6: 34803/1048576,
 7: 23297/1048576,
 8: 7321/524288,
 9: 4359/524288,
 10: 4887/1048576,
 11: 2613/1048576,
 12: 1323/1048576,
 13: 19/32768,
 14: 65/262144,
 15: 117/1048576,
 16: 43/1048576,
 17: 19/1048576,
 18: 3/1048576,
 19: 1/524288,
 20: 1/1048576}

falcon1024dist={-13: 3/1048576,
 -12: 11/524288,
 -11: 109/1048576,
 -10: 307/1048576,
 -9: 1055/1048576,
 -8: 1465/524288,
 -7: 3711/524288,
 -6: 8099/524288,
 -5: 3983/131072,
 -4: 55367/1048576,
 -3: 84393/1048576,
 -2: 113981/1048576,
 -1: 2151/16384,
 0: 72973/524288,
 1: 2151/16384,
 2: 113981/1048576,
 3: 84393/1048576,
 4: 55367/1048576,
 5: 3983/131072,
 6: 8099/524288,
 7: 3711/524288,
 8: 1465/524288,
 9: 1055/1048576,
 10: 307/1048576,
 11: 109/1048576,
 12: 11/524288,
 13: 3/1048576}

In [143]:
B2=CBD(2)
B2.raw_data(228, 228, aborts=False)

n p coresize Eclassic Enoabort Equantum
228 0.507751 458.5966826577156 457.7050598459596 487.75640708516715 225.82021303610733


In [149]:
B1=CBD(1)
B1.raw_data(1023, 1024, aborts=False)

n p coresize Eclassic Enoabort Equantum
1023 0.500000 1529.6628614539638 1528.7646839194656 1574.314497316468 760.989856042805
1024 0.512464 1531.6656289469593 1530.7339832452408 1575.856920919612 762.0009747444512


In [147]:
float(45/1024)

0.0439453125

In [144]:
B2.raw_data(484, 484, aborts=False)

n p coresize Eclassic Enoabort Equantum
484 0.505293 977.9034987030174 976.9801201669309 1039.6055912503793 484.98012709545753


In [142]:
truelog(B2.kc(17).dic[100][1]) < -B2.entropy*17

True

In [91]:
B2.entropy*17

39.66715459110682

In [9]:
2**20

1048576

In [10]:
print("i pismall pibig")
for key in range(21):
    print(key, falcon1024dist.get(key,0/1).numerator()*2**20/falcon1024dist.get(key,0/1).denominator(), falcon512dist.get(key,0/1).numerator()*2**20/falcon512dist.get(key,0/1).denominator())

i pismall pibig
0 145946 104098
1 137664 100876
2 113981 91062
3 84393 77952
4 55367 63141
5 31864 47872
6 16198 34803
7 7422 23297
8 2930 14642
9 1055 8718
10 307 4887
11 109 2613
12 22 1323
13 3 608
14 0 260
15 0 117
16 0 43
17 0 19
18 0 3
19 0 2
20 0 1


In [17]:
print("i pi")
for key in falcon512dist:
    if key>=0:
        print(key, falcon512dist[key].numerator()*2**20/falcon512dist[key].denominator())

i pi
0 104098
1 100876
2 91062
3 77952
4 63141
5 47872
6 34803
7 23297
8 14642
9 8718
10 4887
11 2613
12 1323
13 608
14 260
15 117
16 43
17 19
18 3
19 2
20 1


In [7]:
B1=CBD(1)
B1.make_tablepoints(1,1024,step=1, delete_after=True)

n p coresize Eclassic Equantum
1 0.500000 0.0 0.0 -1.0
2 0.750000 2.321928094887362 1.7004397181410922 0.22994660587693602
3 0.500000 2.807354922057604 2.409390936137702 -0.14425209380333012
4 0.687500 5.044394119358453 4.308339030139407 1.2322580793349238
5 0.500000 5.672425341971496 5.185494924210866 1.097832682453581
6 0.656250 7.86418614465428 7.118697792032526 2.485866395275659
7 0.500000 8.566054038171092 8.049274249337504 2.470370175880328
8 0.636719 10.728770849542665 9.984691030136343 3.82486819429421
9 0.500000 11.477252323937952 10.94232621259836 3.8802943194755
10 0.623047 13.619417596048342 12.875849915387949 5.200497289243783
11 0.500000 14.40054593457218 13.851221495775556 5.305080233481199
12 0.612793 16.527247901386193 15.782825627185055 6.5970519483301295
13 0.500000 17.33276212038744 16.771090138214984 6.7386600653919935
14 0.604736 19.44732743864349 18.701187763533337 8.00774937386243
15 0.500000 20.271876742139984 19.6993051694807 8.178641421572527
16 0.598190 22.3

123 0.500000 181.1138283818195 180.3593583992847 87.91593782851785
124 0.535754 183.1343566969146 182.29712158003144 88.98211929659305
125 0.500000 184.10345363728652 183.34759665354443 89.40343596618591
126 0.535470 186.1236899353753 185.28558538587873 90.46882809277756
127 0.500000 187.09322803974433 186.33600890494282 90.89109283284638
128 0.535193 189.11318060433484 188.27422089696563 91.95571564663167
129 0.500000 190.083147314734 189.3245901133497 92.37890420048147
130 0.534922 192.10282407862653 191.26302309588465 93.44277694270964
131 0.500000 193.07320737057435 192.3133354551242 93.86686601390447
132 0.534658 195.09261593552984 194.25198718264105 94.93000718033238
133 0.500000 196.0634042880128 195.3022403108317 95.35497438125113
134 0.534399 198.08255194305303 197.24110856221643 96.41740176126933
135 0.500000 199.0537343106032 198.29130025382437 96.84322556512586
136 0.534146 201.07262804905608 200.23038283301247 97.90495627830177
137 0.500000 202.0441938357491 201.2805110396

243 0.500000 360.66377598938305 359.8538973444356 177.35448547263374
244 0.525514 362.67480758337035 361.80197888595524 178.3934613845234
245 0.500000 363.6582283978029 362.8477288523925 178.84733035618484
246 0.525410 365.66917587580053 364.7959402460379 179.8860477978781
247 0.500000 366.65272334846054 365.8416091753547 180.34022476418397
248 0.525307 368.6635879888933 367.7899494002899 181.37868728547127
249 0.500000 369.64726019113397 368.8355375558581 181.833167991898
250 0.525206 371.6580432433945 370.784005604393 182.87137906781894
251 0.500000 372.64183829044936 371.82951325387876 183.3261593498825
252 0.525106 374.65254097577343 373.7781081312464 184.36412238285314
253 0.500000 375.63645702543073 374.8235355463018 184.8191981635356
254 0.525007 377.6470805377398 376.7722562704405 185.85691648539932
255 0.500000 378.6311157890671 377.8176037264101 186.3122837726678
256 0.524910 380.6416612957793 379.7664493277489 187.3497606466734
257 0.500000 381.6258139878956 380.811717103392

364 0.520896 542.3969364466037 541.5050461023814 268.0222822476451
365 0.500000 543.385590682332 542.5464542160128 268.4892881397671
366 0.520839 545.3931051605548 544.5009669889006 269.5170903835767
367 0.500000 546.3818187144929 545.5423116760593 269.98425055440856
368 0.520782 548.3892941614209 547.4969098891374 271.01192409858083
369 0.500000 549.3780664151739 548.538191423307 271.4792370729502
370 0.520726 551.3855032350632 550.4928745688061 272.5067831363105
371 0.500000 552.3743335798758 551.5340932221052 272.9742474605245
372 0.520670 554.3817321707221 553.4888607973289 274.0016672443133
373 0.500000 555.3706200072786 554.5300168405035 274.4692814857374
374 0.520615 557.3779807609457 556.4848683477593 275.496576173952
375 0.500000 558.3669254991756 557.5259620501765 275.964338920599
376 0.520560 560.374248801521 559.4808969967054 276.991509680327
377 0.500000 561.3632498604097 560.5219286263472 277.45941954045645
378 0.520506 563.3705360914073 562.4769465242568 278.486467522201

487 0.500000 726.1854780094692 725.3276262268271 359.7191321170147
488 0.518050 728.1911780249219 727.2865736294373 360.74082065232466
489 0.500000 729.1826195342602 728.3245141800453 361.2152320519018
490 0.518013 731.1882970853788 730.2835233185059 362.23684310111423
491 0.500000 732.1797723579166 731.3214148139758 362.711346303891
492 0.517977 734.185427621311 733.280485395041 363.7328804295817
493 0.500000 735.1769363913477 734.3183280268813 364.2074747665155
494 0.517940 737.1825695415507 736.2774597596593 365.22893252506236
495 0.500000 738.1741115465137 737.3152537182388 365.70361733450636
496 0.517904 740.1797227560137 739.2744463141655 366.7249992761812
497 0.500000 741.1712977364094 740.3121917887192 367.1997739037748
498 0.517868 743.1768871756814 742.271444961532 368.2210805728336
499 0.500000 744.1684948750475 743.3091421401695 368.69594437139426
500 0.517832 746.1740627125852 745.2684556058812 369.7171763061653
501 0.500000 747.1657028774432 746.3061046755931 370.19212863

609 0.500000 909.0290470360299 908.1578724500995 451.0041304737241
610 0.516146 911.0336390271978 910.1201219456706 452.02193830946464
611 0.500000 912.0267455478026 911.1553832543653 452.5009442789042
612 0.516120 914.0313229553032 913.1176801179311 453.5187001982842
613 0.500000 915.0244513862784 914.1529022187989 453.99776758998416
614 0.516093 917.0290143025345 916.1152462631396 455.01547189932563
615 0.500000 918.0221645049146 917.1504292906508 455.49460034970167
616 0.516067 920.0267130214727 919.1128203297993 456.5122533525915
617 0.500000 921.0198848576115 920.1479644176795 456.9914425013155
618 0.516041 923.0244190651521 922.110402266909 458.00904449863793
619 0.500000 924.0176123987054 923.145507548144 458.48829398859885
620 0.516015 926.0221323870546 925.1079920239562 459.50584527856796
621 0.500000 927.0153470829645 926.1430586307979 459.9851547558334
622 0.515990 929.0198529411047 928.105589550912 461.00265563402485
623 0.500000 930.0130888655829 929.1406176148835 461.4820

730 0.514760 1090.906377523044 1089.9861736787561 541.8432034402699
731 0.500000 1091.9005909334876 1091.019297426518 542.3253463115523
732 0.514740 1093.9044357985144 1092.9841334197606 543.3404639727885
733 0.500000 1094.8986646515898 1094.0172248730755 543.8226511147064
734 0.514720 1096.9024992900127 1095.9820987449523 544.8377315063194
735 0.500000 1097.8967435034622 1097.0151580017393 545.319962697393
736 0.514700 1099.9005679695738 1098.9800696240663 546.3350060048323
737 0.500000 1100.8948274617956 1100.0130967817556 546.8172810252546
738 0.514680 1102.8986418094573 1101.9780460270836 547.832287432576
739 0.500000 1103.892916499498 1103.0110411826188 548.3146060641969
740 0.514660 1105.896720782144 1104.976027924226 549.3295757540762
741 0.500000 1106.8910105896925 1106.008991174068 549.8119377803854
742 0.514641 1108.8948048603352 1107.9740152859554 550.8268709341324
743 0.500000 1109.8891097057158 1109.0069467260848 551.3092761402427
744 0.514621 1111.892894016949 1110.972008

850 0.513680 1270.7982472249494 1269.8727129945246 631.6901163666367
851 0.500000 1271.7932610873972 1270.9040571146854 632.1745734896922
852 0.513664 1273.796575627041 1272.870961475305 633.1877415167097
853 0.500000 1274.7916009575565 1273.9022784107738 633.672232093281
854 0.513648 1276.7949078956692 1275.8692140819867 634.6853719170842
855 0.500000 1277.7899446415947 1276.900503906342 635.1698958021371
856 0.513632 1279.7932440129803 1278.8674707953094 636.1830075444187
857 0.500000 1280.7882921220196 1279.8987335817985 636.6675645938676
858 0.513616 1282.7915839612438 1281.8657315961468 637.680648375528
859 0.500000 1283.7866433814593 1282.896967417687 638.1652384462271
860 0.513600 1285.7899277228507 1284.8639964655056 639.1782943873816
861 0.500000 1286.7849984026611 1285.8952053946866 639.6629173371175
862 0.513584 1288.7882752803146 1287.8622653845239 640.6759455571028
863 0.500000 1289.7833571684896 1288.8934474936102 641.1606012445856
864 0.513568 1291.7866266162682 1290.860

970 0.512806 1450.7042387327065 1449.774324869475 721.5561889375612
971 0.500000 1451.6998584311932 1450.8041577153954 722.042425410633
972 0.512793 1453.7027712586273 1452.772790867454 723.0540925332447
973 0.500000 1454.698399809109 1453.8026004196806 723.5403551910105
974 0.512780 1456.7013067649596 1455.7712600366917 724.5520002141234
975 0.500000 1457.6969441317174 1456.8010463520613 725.0382889563754
976 0.512767 1459.6998452396167 1458.7697323641876 726.0499119642054
977 0.500000 1460.695491387147 1459.7994954993048 726.5362266913137
978 0.512754 1462.6983866705864 1461.76820783702 727.5478277675934
979 0.500000 1463.6940415636002 1462.797947848259 728.0341683805017
980 0.512740 1465.6969310459285 1464.7666864423475 729.0457476084835
981 0.500000 1466.692594649349 1465.7964033858516 729.5321140087041
982 0.512728 1468.6954783537758 1467.7651681674056 730.5436714711641
983 0.500000 1469.6911506327365 1468.7948620990908 731.0300635607742
984 0.512715 1471.6940285823316 1470.763652

In [4]:
B1=CBD(1)
B1.make_tablepoints(2,176,step=2, delete_after=True)
B1.make_tablepoints(1,176,step=2, delete_after=True)

n p coresize Eclassic Equantum
2 0.750000 2.321928094887362 1.5235619560570128 0.22994660587693602
4 0.687500 5.044394119358453 4.032045726930809 1.2322580793349238
6 0.656250 7.86418614465428 6.8230437518722 2.485866395275659
8 0.636719 10.728770849542665 9.689178591660092 3.82486819429421
10 0.623047 13.619417596048342 12.58439650229453 5.200497289243783
12 0.612793 16.527247901386193 15.496237227349502 6.5970519483301295
14 0.604736 19.44732743864349 18.419565789961077 8.00774937386243
16 0.598190 22.37661745991003 21.35150752513931 9.428850124690577
18 0.592735 25.31310698693521 24.29019974697723 10.857982410200467
20 0.588099 28.255392350293018 27.2343449693568 12.293523697612551
22 0.584094 31.202452699512374 30.182997598447756 13.734309309557394
24 0.580590 34.15352097008393 33.135445086982436 15.179473117637245
26 0.577491 37.108005284614755 36.09113602612717 16.628352624777172
28 0.574723 40.065438774635794 39.04963425296115 18.080429187985377
30 0.572232 43.02544628040058 42.

69 0.500000 100.47381545237899 99.59699154718602 47.836278739027
71 0.500000 103.45662083221038 102.57828169295759 49.31679897840588
73 0.500000 106.43983887337043 105.56004197757917 50.79772898264211
75 0.500000 109.42344980175088 108.54224898939971 52.2790503327734
77 0.500000 112.40743525396881 111.52488104640919 53.76074588227229
79 0.500000 115.3917781446961 114.50791802819442 55.24279963929873
81 0.500000 118.37646254936769 117.49134122797938 56.725196662500615
83 0.500000 121.36147360016284 120.47513322191898 58.20792296851177
85 0.500000 124.34679739348452 123.45927775327304 59.69096544958734
87 0.500000 127.3324209074345 126.44375962946083 61.174311800056415
89 0.500000 130.3183319280064 129.42856463030455 62.657950450469606
91 0.500000 133.30451898290806 132.41367942602565 64.1418705084838
93 0.500000 136.29097128207883 135.39909150376812 65.62606170566401
95 0.500000 139.27767866410048 138.38478910160117 67.11051434949702
97 0.500000 142.26463154781032 141.37076114910056 68.

In [ ]:
n p coresize Eclassic Equantum
2 0.610352 3.1699250014423126 2.3879499938951874 0.30848836763383186
4 0.652909 7.86418614465428 6.59141238494359 2.4462025202558793
6 0.589996 12.031011907437707 10.864102908134283 4.313172950800069
8 0.582214 16.559152996483007 15.392722791647616 6.463966918292766
10 0.562727 20.9986765025611 19.863660994412434 8.573506718977312
12 0.554921 25.534987002894706 24.41431113022643 10.759302513670002
14 0.546340 30.06327048331791 28.95693933504846 12.949689454615289
16 0.540502 34.61792847246424 33.52396909663718 15.163975457510068
18 0.535099 39.17740189313943 38.094608080219565 17.386570002338786
20 0.530583 43.74679150260234 42.674451239049354 19.61961366525679
22 0.526506 48.32169157295572 47.25901371675723 21.8594407822814
24 0.522997 52.90372002355866 51.84956287674342 24.106529272604927
26 0.521170 57.50759852142998 56.45723593108525 26.369020108326875
28 0.523604 62.16902230287224 61.10984440205626 28.667333764405505
30 0.527281 66.84968814952487 65.77881755316314 30.97944275855795
32 0.522937 71.42702392341286 70.36786673691795 33.231065694025276
34 0.529502 76.14983864980486 75.07036199791467 35.57323081447605
36 0.530097 80.79498452834196 79.71325713030889 37.86979599853559
38 0.532310 85.46351378560271 84.37433715166868 40.182185142550864
40 0.531708 90.09628892911257 89.0083579589357 42.47278646640244
42 0.529772 94.71157292984574 93.62906843207911 44.75316949140705
44 0.527390 99.32042699569632 98.2447755154787 47.0304721526058
46 0.524999 103.92874016403046 102.85998226793888 49.3083546633117
48 0.522816 108.53984160755212 107.47734964906037 51.58873525856878
50 0.520769 113.15278212714983 112.0961506609972 53.87094794877911
52 0.518728 117.76544085828041 116.71466222762925 56.153739726873134
54 0.516937 122.38198670758572 121.33630718484271 58.43943115561689
56 0.516032 127.01416523116671 125.97092584048677 60.734347036010355
58 0.516134 131.66507882684016 130.62130781985326 63.04001344820717
60 0.517941 136.34785390456443 135.2985284700802 65.36384697493605
62 0.518806 141.01388265523502 139.96188848915986 67.67932437961522
64 0.517781 145.64538541134783 144.5962295381428 69.97617902687652
66 0.520740 150.34984171019065 149.29175811001895 72.31529485440933
68 0.520754 155.00057369193388 153.94235006991366 74.6245658097973
70 0.522265 159.67904819703793 158.616164522142 76.9505012426981
72 0.522177 164.32891277910147 163.26617093654937 79.26009146583289
74 0.521665 168.9714025019791 167.9100698226536 81.56560485764668
76 0.520375 173.59939202061824 172.5418278081755 83.86295781028967
78 0.518871 178.2229771211759 177.16982539115534 86.15813409359708
80 0.517606 182.85104679739257 181.80158087237652 88.45615628802742
82 0.516407 187.4803634440412 186.43438244165873 90.75511701356713
84 0.515092 192.10704190851763 191.06489079786775 93.05292212160231
86 0.514027 196.73879465266907 195.69972509521045 95.3537739707449
88 0.513840 201.3898707706286 200.3512661994888 97.66517187605375
90 0.513871 206.04603147983175 205.00727334934103 99.97952795966499
92 0.515052 210.72807291654948 209.6857626644252 102.3081042683682
94 0.515379 215.39113570471628 214.34782393374346 104.62696912730456
96 0.514964 220.0375489053145 218.99539709141536 106.93719817647835
98 0.516745 224.73274130742635 223.68526765992982 109.27473906924973
100 0.516697 229.3872114129134 228.33984021799736 111.59026878222475
102 0.517706 234.06523971131836 233.01482119243616 113.9193108106485
104 0.517550 238.71765516734587 237.66766248625083 116.23418344794165
106 0.517427 243.37106477 242.32138495887057 118.54973552708734
108 0.516635 248.009510030073 246.96214336239626 120.85700525005471
110 0.515630 252.64293129905911 251.59851267608468 123.16164222852989
112 0.514855 257.28167076113857 256.23951081910985 125.46933104088414
114 0.514110 261.9211862604191 260.8811941586444 127.77752061283086
116 0.513213 266.55690096551024 265.5195287663085 130.08380232702947
118 0.512525 271.1977152705574 270.162338431626 132.3929427402205
120 0.512546 275.856557788047 274.8210788170954 134.71169511191923
122 0.512449 280.51254132933497 279.4773235898847 137.02910292090982
124 0.513249 285.1915894551608 284.1539873597247 139.358916724496
126 0.513392 289.8537490679502 288.8157103635346 141.68009342495125
128 0.513156 294.5061114815861 293.46873627829143 143.99629918507748
130 0.514358 299.19515352951146 298.1542085891814 146.33261327944444
132 0.514297 303.85181991637774 302.81103405370914 148.65170971683258
134 0.514998 308.5279528803976 307.4850724470511 150.98166994396118
136 0.514790 313.1809845129708 312.13870328736044 153.2991077819339
138 0.514783 317.83934379124787 316.7970548789554 155.61954368298183
140 0.514267 322.4846193465562 321.4438354095834 157.9328708610814
142 0.513571 327.12514297665996 326.08639669101524 160.2436877264168
144 0.513086 331.77129717362965 330.73396222928454 162.55759646009875
146 0.512612 336.41788926559445 335.38192992162567 164.87175863567845
148 0.511976 341.0600037057038 340.02590135782424 167.18361131866746
150 0.511513 345.70692345494734 344.6741638130982 169.4980713666495
152 0.511591 350.36919345212846 349.3361834325319 171.82063126070358
154 0.511432 355.0247994436265 353.9922479364837 174.13981744542338
156 0.511990 359.7009209509438 358.6667148494029 176.46990099853113
158 0.512049 364.3627583968013 363.3283670967473 178.79269524239842
160 0.511875 369.0178102164024 367.9839164744114 181.11208099497844
162 0.512736 373.7023438611276 372.6659023420426 183.4473906265475
164 0.512672 378.36040643729154 377.3241411928939 185.76877233446916
166 0.513174 383.0346145434295 381.99685640503 188.0989979542732
168 0.512953 387.6882312316754 386.6511165763075 190.41822156303033
170 0.512981 392.3491059597176 391.3118917287889 192.74141101687226
172 0.512633 396.9991737304699 395.962974555364 195.05881589879877
174 0.512134 401.6448728703166 400.61013207622915 197.37391555551545
176 0.511825 406.2962593441558 405.26241232428 199.69206716786857

n p coresize Eclassic Equantum
1 0.781250 1.5849625007211563 0.9307373375628863 0.07025977040248076
3 0.531769 5.044394119358453 4.216638412152119 0.901877500917815
5 0.595022 9.802516365121223 8.67204673885272 3.2617347286171694
7 0.544475 14.026782772345964 12.98093695888166 5.17366867706581
9 0.543700 18.568991482546537 17.511671799706193 7.362746699571316
11 0.529526 23.03312368391756 21.993875075746818 9.495814841107569
13 0.525996 27.587165971534485 26.549829830339863 11.7042374104512
15 0.530653 32.23189919245517 31.169488410788166 13.97532543590847
17 0.545173 36.97852583106552 35.86686131058827 16.322666887568012
19 0.550938 41.64991900151769 40.51607716106834 18.62407204294371
21 0.548697 46.25051799898636 45.12037612474262 20.87891312243603
23 0.543912 50.82740623572464 49.71003906659655 23.119893984260422
25 0.539588 55.408860288625796 54.30323838861123 25.366580793130545
27 0.536334 60.00223616776822 58.905175852690036 27.623306498752314
29 0.532878 64.59355839901475 63.50599874851945 29.880533075490995
31 0.529630 69.18726705512964 68.10863090766892 32.14102764694916
33 0.526608 73.78365413591317 72.71339618232474 34.40478207887428
35 0.523805 78.38271718734332 77.32019955254529 36.671618672823
37 0.521343 82.98639548255227 81.93066019900019 38.94250069264866
39 0.519858 87.60473925103544 86.55284753443826 41.222911136573636
41 0.518190 92.22079617779876 91.17339780358209 43.5030027444636
43 0.516118 96.83026071199443 95.78858442878418 45.78057496652322
45 0.517874 101.50204043865426 100.4546772468282 48.09386032240751
47 0.520273 106.18496196357518 105.13018189533769 50.414984340418066
49 0.524321 110.894421048991 109.82729692114528 52.75341063392468
51 0.526608 115.57630394765262 114.50204926672204 55.07733098840852
53 0.527945 120.24403985895721 119.16541355676516 57.39377648712143
55 0.526408 124.86723452241165 123.79303034385428 59.68368486151122
57 0.524619 129.48599323850164 128.4169929225599 61.97162051894333
59 0.523832 134.1213362975917 133.05449392203036 64.26995852152405
61 0.522230 138.74316160709162 137.68096782632404 66.56066615418167
63 0.520564 143.3636778652097 142.30632814143348 68.85106956306367
65 0.518853 147.98309003357116 146.9307170669425 71.14131411673816
67 0.517331 152.60568514287777 151.5577172577321 73.43379827760542
69 0.516103 157.23379885643166 156.18934790216284 75.72971559262764
71 0.515754 161.87942414627105 160.83587251887042 78.03544264465994
73 0.514859 166.51435909299929 165.47335194423752 80.33573670828706
75 0.513919 171.1481149928148 170.10977408601744 82.63589100357757
77 0.515407 175.83203951506695 174.78918367716136 84.96350339793172
79 0.516250 180.50266167490548 179.45725459727623 87.28471760166919
81 0.518457 185.2009898229264 184.14896119957427 89.62212413246351
83 0.519369 189.87300327354265 188.8182087289135 91.94559385204913
85 0.520563 194.5507875749916 193.4923287370952 94.27296072583252
87 0.519824 199.18994319473398 198.13361810006194 96.57843826871083
89 0.518827 203.82365221814004 202.7702367002011 98.88112145550511
91 0.518592 208.47321212665122 207.42042451929754 101.19303873384317
93 0.517593 213.10693073705082 212.05706407821117 103.49610963941008
95 0.516576 217.74024113899927 216.69334196468276 105.79912106060809
97 0.515489 222.37189977872788 221.32817595064222 108.1014026445936
99 0.514563 227.0070639846622 225.96603379446742 110.40576869025217
101 0.513785 231.64564458870015 230.60686570100734 112.7121220825244
103 0.513723 236.30119844603595 235.26254949552703 115.02762419100667
105 0.513148 240.94472053723993 239.90772633040444 117.33693221643863
107 0.512615 245.58906134484522 244.55359354480268 119.64694650842401
109 0.513698 250.2724967834023 249.23378825766406 121.97797859050583
111 0.513994 254.93679883467007 253.89719942354222 124.29928045951392
113 0.515435 259.6284838745564 258.5845924112858 126.63587102673306
115 0.515779 264.2938412262879 263.248916865217 128.9585451996954
117 0.516683 268.97252981986685 267.9248804455399 131.28895134277215
119 0.516242 273.6192909789295 272.5729188217676 133.6018159198505
121 0.515610 278.26138057644187 277.2168524149468 135.91226864976616
123 0.515559 282.91768073161506 281.87326636774964 138.23069237607533
125 0.514873 287.5585570100104 286.51614921751195 140.54066748796464
127 0.514210 292.2000148323888 291.15954378083876 142.85103034659087
129 0.513502 296.8403795870438 295.80197273328355 145.16087305006369
131 0.512909 301.4837087013131 300.4470265094264 147.4723707485988
133 0.512366 306.1283941871653 305.0932905460324 149.78465499592286
135 0.512397 310.7885326438765 309.7533101985234 152.10513359480376
137 0.511997 315.4371609885756 314.4030961561432 154.41968585004113
139 0.511641 320.08690112432095 319.05385820884817 156.73499173607908
141 0.512433 324.76785605076464 323.73246025433724 159.06690970480562
143 0.512490 329.42869790445366 328.39312826002106 161.38850435211177
145 0.513515 334.1157240626284 333.0771158266682 163.72436130396918
147 0.513596 338.77714705020253 337.7382919987233 166.04675666898848
149 0.514270 343.45452071144416 342.41365474692503 168.3780221418906
151 0.513965 348.10551633140363 347.0655322582204 170.69508738844925
153 0.513534 352.7530037171152 351.71427712101075 173.01035753408604
155 0.513558 357.4130619210361 356.374244474939 175.33251377442897
157 0.513068 362.0590492392152 361.02166373801816 177.64708214357788
159 0.512614 366.7061032854409 365.67004006144623 179.96225489266624
161 0.512143 371.3527381761371 370.31804575940345 182.27723018244916
163 0.511747 376.0015751232986 374.9680339327465 184.59339663776987
165 0.511351 380.6504869689447 379.618097247444 186.90963638731682
167 0.511414 385.3130481902468 384.2804559939467 189.23305065112655
169 0.511124 389.9651642706557 388.93341399342467 191.55109399591564
171 0.510859 394.61797972528046 393.58699455017046 193.86961626026167
173 0.511449 399.29639624508985 398.2636639979963 196.20166864190097
175 0.511390 403.9552760505561 402.92271409714374 198.52367070706612

In [4]:
def approx_falcon_gaussian(n,key_samples):
    dist={}
    for i in range(key_samples):
        clear_output(wait=True)
        print(i)
        sample=sample_gaussian(n)
        for val in sample:
            dist[abs(val)]=dist.get(abs(val),0)+1/(n*key_samples)
    keylist1=[]
    for key in dist:
        keylist1+=[key]
    keylist1.sort(reverse=True)
    if keylist1[len(keylist1)-1]==0:
        keylist1=keylist1[:len(keylist1)-1]
    keylist2=keylist1.copy()
    keylist2.sort()
    out={}
    for key in keylist1:
        out[-key]=dist[key]/2
    if 0 in dist:
        out[0]=dist[0]
    for key in keylist2:
        out[key]=out[-key]
    return out

In [44]:
testfalcon512=approx_falcon_gaussian(512,2**10)
testfalcon1024=approx_falcon_gaussian(1024,2**9)

511


In [5]:
falcon512=distributions(falcon512dist)
falcon1024=distributions(falcon1024dist)
gc.collect()

91

In [55]:
B2
n p coresize Eclassic Equantum
1 0.875000 1.5849625007211563 0.8073549220576041 0.21607124505355713
2 0.765625 3.1699250014423126 2.076815597050831 0.5997442720007307
3 0.669922 4.754887502163469 3.654188473059545 1.0849045981735481
4 0.586182 6.339850002884625 5.3510254810709545 1.6203829131112217
5 0.525269 7.98299357469431 7.091272664849976 2.238294642353228
6 0.491436 9.749869427396844 8.898936542567105 2.9801827241080816
7 0.479484 11.647908074281808 10.803660855589229 3.84578126413115
8 0.595192 14.397741472348816 13.242105499355647 5.418165484347093
9 0.587829 16.33436058750693 15.194302769002038 6.3447888804101895
10 0.573746 18.229236598251877 17.121425067488815 7.237098301390574
11 0.556222 20.099539159061763 19.031286385562716 8.109263806894159
12 0.538751 21.96513076769832 20.9348299938242 8.979012267661354
13 0.523773 23.84291856109702 22.844166514891366 9.860860006895374
14 0.512304 25.742828138080977 24.76843279972802 10.762627658492192
15 0.504229 27.667004075358324 26.710880763766394 11.685433161080823
16 0.560011 30.14793899791188 29.03590083468342 13.011583218629065
17 0.553110 32.095811624702684 31.002908536693898 13.951662938054948
18 0.545054 34.031984727238715 32.959929825371354 14.88417306489989
19 0.536477 35.961100836152205 34.91003711898939 15.812656047944515
20 0.528045 37.88854095436984 36.85741325503015 16.741273542422437
21 0.520289 39.819232546179144 38.806284492904965 17.673684256933726
22 0.513533 41.75690299188287 40.76004442291052 18.612420930748
23 0.507904 43.70381178198096 42.720781632017285 19.55876065454969
24 0.544984 46.09429417631799 45.00589630839765 20.805237109049568
25 0.539865 48.05525940148478 46.98105089529929 21.76018595729695
26 0.534343 50.01066005833099 48.951181016518014 22.71203765735756
27 0.528664 51.962678667365104 50.91785577873921 23.662383554404386
28 0.523082 53.91382471763711 52.88310416234488 24.613002810320506
29 0.517803 55.86643330340365 54.84894976928681 25.565468224902993
30 0.512989 57.8226140491614 56.81725042714117 26.521104198701515
31 0.508742 59.78398569237755 58.789431708839494 27.480823909771953
32 0.536618 62.125060137826395 61.05074523526267 28.686235684413464
33 0.532652 64.0953920202629 63.03206410544377 29.651238975788214
34 0.528478 66.06223041210302 65.01021279993593 30.614435140877372
35 0.524225 68.02693404365833 66.98621432833293 31.576764660295147
36 0.520061 69.99143500775357 68.96160736621644 32.53951198885203
37 0.516054 71.95667586424 70.93726337440287 33.503257065280856
38 0.512306 73.92404300463345 72.91437179187648 34.468857085574165
39 0.508903 75.89491631647881 74.8941104285629 35.43710009942912
40 0.531251 78.20310894650318 77.13815959567509 36.616645905522844
41 0.528027 80.18007923410774 79.12409132621366 37.58851623290667
42 0.524670 82.15454446800442 81.10774962878963 38.559130604784905
43 0.521260 84.12743306086449 83.08985767253795 39.52910446266807
44 0.517985 86.10122582036641 85.07236695991783 40.50001337039015
45 0.514779 88.07517506648286 87.05481777753769 41.47132298581264
46 0.511695 90.04999508835374 89.03781938406262 42.44351121040328
47 0.508848 92.02745266269548 91.02282438994992 43.41760644512159
48 0.527493 94.31146418492001 93.25324031881837 44.57902775126249
49 0.524769 96.2932690807075 95.24264823408284 45.55579577597914
50 0.521951 98.27311907416302 97.23028252357605 46.53159208749638

B3
n p coresize Eclassic Equantum
1 0.781250 1.5849625007211563 0.9307373375628863 0.07025977040248076
2 0.610352 3.1699250014423126 2.3879499938951874 0.30848836763383186
3 0.531769 5.044394119358453 4.216638412152119 0.901877500917815
4 0.652909 7.86418614465428 6.59141238494359 2.4462025202558793
5 0.595022 9.802516365121223 8.67204673885272 3.2613375301864354
6 0.589996 12.031011907437707 10.864102908134283 4.312976598069562
7 0.544475 14.026782772345964 12.98093695888166 5.1735618061568
8 0.582214 16.559152996483007 15.392722791647616 6.463941884176219
9 0.543700 18.568991482546537 17.511671799706193 7.362739757440529
10 0.562727 20.9986765025611 19.863660994412434 8.573505175830826
11 0.529526 23.03312368391756 21.993875075746818 9.495814418025818
12 0.554921 25.534987002894706 24.41431113022643 10.759302426220142
13 0.525996 27.587165971534485 26.549829830339863 11.704237388361364
14 0.546340 30.06327048331791 28.95693933504846 12.949689450038743
15 0.530653 32.23189919245517 31.169488410788166 13.975325434819895
16 0.540502 34.61792847246424 33.52396909663718 15.163975457277735
17 0.545173 36.97852583106552 35.86686131058827 16.32266688751751
18 0.535099 39.17740189313943 38.094608080219565 17.386570002327126
19 0.550938 41.64991900151769 40.51607716106834 18.62407204294131
20 0.530583 43.74679150260234 42.674451239049354 19.61961366525621
21 0.548697 46.25051799898636 45.12037612474262 20.87891312243591
22 0.526506 48.32169157295572 47.25901371675723 21.85944078228137
23 0.543912 50.82740623572464 49.71003906659655 23.119893984260415
24 0.522997 52.90372002355866 51.84956287674342 24.106529272604927
25 0.539588 55.408860288625796 54.30323838861123 25.36658079313054
26 0.521170 57.50759852142998 56.45723593108525 26.369020108326875
27 0.536334 60.00223616776822 58.905175852690036 27.623306498752314
28 0.523604 62.16902230287224 61.10984440205626 28.667333764405505
29 0.532878 64.59355839901475 63.50599874851945 29.880533075490995
30 0.527281 66.84968814952487 65.77881755316314 30.97944275855795
31 0.529630 69.18726705512964 68.10863090766892 32.14102764694916
32 0.522937 71.42702392341286 70.36786673691795 33.231065694025276
33 0.526608 73.78365413591317 72.71339618232474 34.40478207887428
34 0.529502 76.14983864980486 75.07036199791467 35.57323081447605
35 0.523805 78.38271718734332 77.32019955254529 36.671618672823
36 0.530097 80.79498452834196 79.71325713030889 37.86979599853559
37 0.521343 82.98639548255227 81.93066019900019 38.94250069264866
38 0.532310 85.46351378560271 84.37433715166868 40.182185142550864
39 0.519858 87.60473925103544 86.55284753443826 41.222911136573636
40 0.531708 90.09628892911257 89.0083579589357 42.47278646640244
41 0.518190 92.22079617779876 91.17339780358209 43.5030027444636
42 0.529772 94.71157292984574 93.62906843207911 44.75316949140705
43 0.516118 96.83026071199443 95.78858442878418 45.78057496652322
44 0.527390 99.32042699569632 98.2447755154787 47.0304721526058
45 0.517874 101.50204043865426 100.4546772468282 48.09386032240751
46 0.524999 103.92874016403046 102.85998226793888 49.3083546633117
47 0.520273 106.18496196357518 105.13018189533769 50.414984340418066
48 0.522816 108.53984160755212 107.47734964906037 51.58873525856878
49 0.524321 110.894421048991 109.82729692114528 52.75341063392468
50 0.520769 113.15278212714983 112.0961506609972 53.87094794877911

Falcon1024
n p coresize Eclassic Equantum
1 0.619873 2.321928094887362 1.6862235116772097 0.019433462564931527
2 0.612995 5.614709844115208 4.518551585850043 1.3965090917728873
3 0.592664 8.92184093707449 7.711725403594062 2.905814597928823
4 0.581754 12.3063465536885 11.080861686483876 4.502895542033113
5 0.581568 15.774298049690474 14.531986566866081 6.171652702156798
6 0.579507 19.247883485213144 18.005797986277404 7.847896344426782
7 0.572693 22.700856014841396 21.478909721955745 9.511014228819374
8 0.566154 26.160134573701033 24.958425620492616 11.182421187422584
9 0.561318 29.635361402160186 28.448648025591233 12.868468690844887
10 0.558726 33.131770171598035 31.952933668753587 14.571985378386717
11 0.558821 36.65425763520127 35.47471500335884 16.295771678298962
12 0.555269 40.14866049008663 38.98041044366533 18.0019408844378
13 0.551451 43.64192358911623 42.48581300290762 19.709182952505024
14 0.548341 47.14269911133948 45.99642412566794 21.423125410807863
15 0.548052 50.672660696942295 49.52721643151869 23.157560916430786
16 0.547673 54.20372906136014 53.059425756950496 24.893930002112906
17 0.546142 57.72445847809079 56.58497717006412 26.624701306438048
18 0.543739 61.23675462314562 60.10485200735161 28.351114365273318
19 0.542075 64.75777268857216 63.631088208236925 30.084036039350273
20 0.541655 68.2937960263442 67.1684104317531 31.827226181796522
21 0.541315 71.831791948782 70.70745951079786 33.57247703436642
22 0.540403 75.36393925927364 74.24245725268572 35.314845859386445
23 0.538840 78.88881260052368 77.77221531955233 37.0534130565009
24 0.537633 82.41849226688713 81.30565694637981 38.79558085128923
25 0.537150 85.95789480342688 84.84655795424918 40.54426981341946
26 0.537043 89.50281755522343 88.39180638812103 42.29689693085851
27 0.536333 93.04043615699113 91.93162963992908 44.04565862223568
28 0.535230 96.5731885446456 95.46780699798065 45.79199568940682
29 0.534396 100.10994293875805 99.0071426383863 47.54119231878175
30 0.534811 103.68605473957143 102.58196535663735 49.3135967367679
31 0.533925 107.22357847501027 106.12223297967499 51.0640938788581
32 0.532937 110.75989434436927 109.66160035380803 52.814273366392975
33 0.532471 114.30408701637894 113.20722859577076 54.56945263959033
34 0.532410 117.85461628069463 116.7579447123747 56.3287271869438
35 0.532103 121.40181470878365 120.30608890830231 58.086433047001485
36 0.531368 124.94280132377801 123.8493376413764 59.840846196880754

Falcon512
n p coresize Eclassic Equantum
1 0.734482 3.1699250014423126 2.199027661318364 0.5845573301788753
2 0.624448 6.6582114827517955 5.463626148562874 1.9257168886016338
3 0.602217 10.470658874060552 9.19370094095022 3.6930223091123158
4 0.592471 14.368028939916165 13.086897105659459 5.5502695230608685
5 0.583693 18.289059768343062 17.029332728344723 7.43142300342672
6 0.576450 22.230921381227503 20.993429520808945 9.332659253097578
7 0.571039 26.19253710910609 24.972403745561028 11.252593348366858
8 0.566057 30.162965936510222 28.959061199983154 13.182253780722073
9 0.562462 34.14822352948862 32.956005719711776 15.125434552828578
10 0.559307 38.14079211745363 36.958823350800714 17.076166215867527
11 0.556465 42.13900968375968 40.966246678655644 19.032955570012163
12 0.554186 46.14467970406299 44.9792697325767 20.996725056962994
13 0.551884 50.15231374312386 48.994311035968465 22.963494264857946
14 0.550049 54.1661770124721 53.014053732632824 24.935920490163504
15 0.548400 58.18352935917824 57.03667090243166 26.91199077336968
16 0.546837 62.20318935914503 61.06130935349931 28.890787678644926
17 0.545444 66.2258810828261 65.08842562042139 30.872658893497356
18 0.544169 70.2509850442624 69.11756565750638 32.85709486825777
19 0.542974 74.27798300150947 73.1483377555089 34.84367303083246
20 0.541881 78.30704506301589 77.18084393496645 36.83241882638797
21 0.540881 82.33801446096994 81.21495872137194 38.823159484936
22 0.539944 86.37050084097368 85.25038464560356 40.815587456312684

105

In [ ]:
falcon512.make_tablepoints(23,27,delete_after=False)

n p coresize Eclassic Equantum


In [15]:
falcon512.kcl

{16: List with 326833 clusters, with probability at least 2^(-H(.)n-0), total probability is 0.0}

In [13]:
B2=CBD(2)
B2.make_tablepoints(16,16,delete_after=False)
B2.kcl

n p_til_entropy size_til_entropy ELZ Equantum
16 0.560011 30.14793899791188 29.03590083468342 13.011583223336187


{16: List with 43 clusters, with probability at least 2^(-H(.)n-0), total probability is 0.0}

In [7]:
falcon512.dist

{-20: 1/1048576,
 -19: 1/524288,
 -18: 3/1048576,
 -17: 19/1048576,
 -16: 43/1048576,
 -15: 117/1048576,
 -14: 65/262144,
 -13: 19/32768,
 -12: 1323/1048576,
 -11: 2613/1048576,
 -10: 4887/1048576,
 -9: 4359/524288,
 -8: 7321/524288,
 -7: 23297/1048576,
 -6: 34803/1048576,
 -5: 187/4096,
 -4: 63141/1048576,
 -3: 609/8192,
 -2: 45531/524288,
 -1: 25219/262144,
 0: 52049/524288,
 1: 25219/262144,
 2: 45531/524288,
 3: 609/8192,
 4: 63141/1048576,
 5: 187/4096,
 6: 34803/1048576,
 7: 23297/1048576,
 8: 7321/524288,
 9: 4359/524288,
 10: 4887/1048576,
 11: 2613/1048576,
 12: 1323/1048576,
 13: 19/32768,
 14: 65/262144,
 15: 117/1048576,
 16: 43/1048576,
 17: 19/1048576,
 18: 3/1048576,
 19: 1/524288,
 20: 1/1048576}

In [8]:
falcon1024.dist

{-13: 3/1048576,
 -12: 11/524288,
 -11: 109/1048576,
 -10: 307/1048576,
 -9: 1055/1048576,
 -8: 1465/524288,
 -7: 3711/524288,
 -6: 8099/524288,
 -5: 3983/131072,
 -4: 55367/1048576,
 -3: 84393/1048576,
 -2: 113981/1048576,
 -1: 2151/16384,
 0: 72973/524288,
 1: 2151/16384,
 2: 113981/1048576,
 3: 84393/1048576,
 4: 55367/1048576,
 5: 3983/131072,
 6: 8099/524288,
 7: 3711/524288,
 8: 1465/524288,
 9: 1055/1048576,
 10: 307/1048576,
 11: 109/1048576,
 12: 11/524288,
 13: 3/1048576}

In [56]:
falcon1024.kcl

{}

In [9]:
falcon1024.kcl

{}

In [329]:
low_n=1
high_n=33
delete_after=False
print("n entropy*n p_til_entropy size_til_entropy ELZ")
for n in range(low_n, high_n+1):
    L=falcon1024.sam_parlist(n, c=0, if_ret=True, if_print=False)
    p=0
    count=0
    ELZ=0
    for l in L.dic:
        p+=l[1]*l[2]
        ELZ+=l[1]*(count+l[2]*(l[2]+1)/2)
        count+=l[2]
    ELZ+=(1-p)*count
    print(n,n*falcon1024.entropy,format(float(p),'f'),truelog(count),truelog(ELZ))
    if delete_after:
        del(self.kcl[n])
        gc.collect()

n entropy*n p_til_entropy size_til_entropy ELZ
1 3.56446738855457 0.620302 2.321928094887362 1.6859532848139176
2 7.12893477710914 0.614170 5.614709844115208 4.515466399096176
3 10.69340216566371 0.593722 8.92184093707449 7.7083375154173455
4 14.25786955421828 0.582818 12.3063465536885 11.077307724633668
5 17.82233694277285 0.581276 15.766037091034477 14.524788652717797
6 21.38680433132742 0.578084 19.232226504773255 17.995048464525492
7 24.95127171988199 0.573685 22.700856014841396 21.475568904705067
8 28.51573910843656 0.567166 26.160108715845645 24.955039659419242
9 32.08020649699113 0.561871 29.631548356849933 28.443023243433213
10 35.6446738855457 0.558964 33.124967394161686 31.94535333328204
11 39.20914127410027 0.558296 36.640587292999186 35.46275804323453
12 42.77360866265484 0.555883 40.144423461268126 38.97417921603975
13 46.338076051209406 0.552350 43.63999227917909 42.48099026482767
14 49.90254343976398 0.549053 47.13871478440452 45.99016440583863
15 53.46701082831855 0.547

KeyboardInterrupt: 

In [99]:
for e in range(1,15):
    n=2**e
    print((B1.count_til_p(n,0)-B1.entropy*n-1)/truelog(n))

-1.6780719051126378
-0.9778029403207733
-0.7570763834857784
-0.6558456350224926
-0.6024556959772951
-0.5725781514115363
-0.555259913666451
-0.544792338027591
-0.5380728224584901
-0.5334371053040741
-0.5300094815667183
-0.5273281412947503
-0.5251419847744938
-0.5233069763882148


In [21]:
s=0
for i in B3.kc(200).dic:
    s+=i[1]*i[2]
float(s)

0.6614031728023684

In [10]:
from operator import itemgetter

from pympler import tracker

mem = tracker.SummaryTracker()
print(sorted(mem.create_summary(), reverse=True, key=itemgetter(2))[:10])

[['str', 153005, 40814256], ['dict', 36380, 15131800], ['code', 44877, 6510960], ['type', 5589, 5088336], ['tuple', 39733, 3148328], ['list', 28115, 3104672], ['set', 1322, 1146384], ['sage.misc.classcall_metaclass.ClasscallMetaclass', 903, 969208], ['weakref', 9505, 836440], ['method_descriptor', 8824, 705920]]


In [11]:
B3=CBD(3)
B2=CBD(2)

In [ ]:
B2.make_tablepoints(1,50,delete_after=True)
B3.make_tablepoints(1,50,delete_after=True)
falcon1024.make_tablepoints(30,50,delete_after=True)
falcon512.make_tablepoints(1,50,delete_after=True)

n p_til_entropy size_til_entropy ELZ Equantum
1 0.875000 1.5849625007211563 0.8073549220576041 0.21607124505355713
2 0.765625 3.1699250014423126 2.076815597050831 0.5997442720007307
3 0.669922 4.754887502163469 3.654188473059545 1.0849045981735481
4 0.586182 6.339850002884625 5.3510254810709545 1.6203829131112217
5 0.525269 7.98299357469431 7.091272664849976 2.238294642353228
6 0.491436 9.749869427396844 8.898936542567105 2.9801827241080816
7 0.479484 11.647908074281808 10.803660855589229 3.84578126413115
8 0.595192 14.397741472348816 13.242105499355647 5.418165484347093
9 0.587829 16.33436058750693 15.194302769002038 6.3447888804101895
10 0.573746 18.229236598251877 17.121425067488815 7.237098301390574
11 0.556222 20.099539159061763 19.031286385562716 8.109263806894159
12 0.538751 21.96513076769832 20.9348299938242 8.979012267661354
13 0.523773 23.84291856109702 22.844166514891366 9.860860006895374
14 0.512304 25.742828138080977 24.76843279972802 10.762627658492192
15 0.504229 27.6670

In [185]:
comb(35,3)

6545.0

In [17]:
L=parlists(1,[],CBD(3).d,1)

In [102]:
float(3/2048)

0.00146484375

In [48]:
Torben={((1,2),(3,4), 2): 1}
Torben

{((1, 2), (3, 4), 2): 1}

In [4]:
def true_runtime(eta,n,p,l=True, cluster=False):
    L=sam_list(eta,n,ret=True)
    cp=0
    i=0
    time=0
    while cp<p:
        cp+=L[i][1]*L[i][2]
        time+=L[i][2]
        i+=1
    if l:
        time=truelog(time)
    if cluster:
        return i-1
    return time

def true_runtime_likelihood(eta,n,p,l=True, cluster=False):
    L=sam_list_p(eta,n,ret=True)
    cp=0
    i=0
    time=0
    while cp<p:
        cp+=L[i][1]
        time+=L[i][3]
        i+=1
    if l:
        time=truelog(time)
    if cluster:
        return i-1
    return time

def true_size_coreset(eta,n,eps, l=True):
    L=sam_list_p(eta, n,ret=True)
    p0=2**(-n*(quentropy(w[eta],1)+eps))
    count=0
    for l in L:
        if l[2]<p0:
            break
        else:
            count+=l[3]
    if l:
        count=truelog(count)
    return count

In [5]:
def full_runtime(eta,n):
    sam_list(eta,n)
    L=key_cluster[eta,n]
    count=0
    E=0
    for i in range(len(L)):
        count+=L[i][1]*(E*L[i][2]+L[i][2]*(L[i][2]+1)//2)
        E+=L[i][2]
    return count

def runtime_keyenum(eta, n, epsilon):
    global key_cluster
    global w
    sam_list(eta,n)
    L=key_cluster[eta,n]
    dist=w[eta]
    count=0
    E=0
    p=2**(-n*(quentropy(dist,1)+epsilon))
    i=0
    while i<len(L) and L[i][1]>=p:
        count+=L[i][1]*(E*L[i][2]+L[i][2]*(L[i][2]+1)//2)
        E+=L[i][2]
        i+=1
    return count

def succprob_lower(eta, n, eps):
    global w
    dist=w[eta]
    c=log(min(dist),2)**2/2
    return 1-exp(-n*eps**2/c)

def batch_succprob_lower(eta, n, eps1):
    global w
    dist=w[eta]
    c=log(min(dist),2)**2/2
    eps=[eps1, (c*log(n)/n)**(1/2), c**(1/2)/n, low_eps(eta,n,1/2)]
    return [1-exp(-n*ep**2/c) for ep in eps]

def runtime_upper_keyenum(eta, n, eps):
    global w
    dist=w[eta]
    c=log(min(dist),2)**2/2
    return n*(quentropy(dist,1)+eps)

def batch_runtime_upper_keyenum(eta, n, eps1):
    global w
    dist=w[eta]
    c=log(min(dist),2)**2/2
    eps=[eps1, (c*log(n)/n)**(1/2), c**(1/2)/n, low_eps(eta,n,1/2)]
    return [n*(quentropy(dist,1)+ep) for ep in eps]

def batch_runtime_keyenum(eta, n, eps1):
    global w
    dist=w[eta]
    c=log(min(dist),2)**2/2
    eps=[eps1, (c*log(n)/n)**(1/2), c**(1/2)/n, low_eps(eta,n,1/2)]
    return [flog(runtime_keyenum(eta, n,ep)) for ep in eps]

In [48]:
def howmany_top(eta,n,epsilon_success, ifprint=True):
    sam_list(eta,n)
    L=key_cluster[eta,n]
    count=0
    p_count=0
    i=0
    while p_count<epsilon_success:
        p_count+=L[i][1]*L[i][2]
        count+=L[i][2]
        i+=1
    if count>0:
        logcount=truelog(count)
    else:
        logcount=count
    if ifprint:
        print("look through ~2^" + str(int(ceil(logcount))) + " possible keys in " + str(i) + " many clusters")
    return(logcount, i)

def howhigh_prob(eta,n,top):
    sam_list(eta,n)
    L=key_cluster[eta,n]
    count=0
    p_count=0
    i=0
    while i<top:
        p_count+=L[i][1]*L[i][2]
        count+=L[i][2]
        i+=1
    if count>0:
        logcount=truelog(count)
    else:
        logcount=count
    print("contains ~2^" + str(int(ceil(logcount))) + " many keys, has success probability " + str(float(p_count)))

def howhigh_runtime(eta,n,limit):
    sam_list(eta,n)
    L=key_cluster[eta,n]
    count=0
    p_count=0
    i=0
    while count<limit:
        p_count+=L[i][1]*L[i][2]
        count+=L[i][2]
        i+=1
    print("look through " + str(i) + " many clusters, find key with probability " + str(float(p_count)))
    
def count_til_entropy(eta,n,ret_logcount=False,ret_count=False,ret_i=False,remove_overhead=True,fake_overhead=False):
    L=sam_list(eta,n,ret=True)
    UB=(quentropy(w[eta],1)*n)
    i=0
    p=0
    count=0
    while count==0 or truelog(count)<UB:
        l=L[i]
        count+=l[2]
        p+=l[1]*l[2]
        i+=1
    if fake_overhead:
        fake_overhead=count-floor(UB)
        if not(isfinite(2**UB)):
            overhead=count-2**(floor(UB))
        else:
            overhead=count-floor(2**UB)
        fake_p=p-fake_overhead*l[1]
        p=p-overhead*l[1]
        return(float(p),float(fake_p))
    if remove_overhead:
        if not(isfinite(2**UB)):
            overhead=count-2**(floor(UB))
        else:
            overhead=count-floor(2**UB)
        count=count-overhead
        p=p-overhead*l[1]
    if ret_count:
        return (float(p),count)
    if ret_logcount:
        return (float(p),truelog(count))
    if ret_i:
        return (float(p),i)
    return float(p)

In [52]:
def tablepoints(eta,low,high,eps1):
    global w
    print("n expected entropy eps1 (c*log(n)/n)**(1/2) c**(1/2)/n low_eps true_eps1 true_(c*log(n)/n)**(1/2) true_c**(1/2)/n true_low_eps")
    scale={}
    for n in range(low, high+1):
        scale[n]=[flog(full_runtime(eta,n)), quentropy(w[eta],1)*n] + batch_runtime_upper_keyenum(eta, n, eps1) + batch_runtime_keyenum(eta, n, eps1) + [howmany_top(eta,n,0.5,ifprint=False)[0]]
        print(n,scale[n][0],scale[n][1],scale[n][2],scale[n][3],scale[n][4],scale[n][5],scale[n][6],scale[n][7],scale[n][8],scale[n][10])

def tablepoints_v2(eta,low,high):
    global w
    print("n entropy true_1n UB_1n true_12 UB_12 true_34 UB_34")
    entr=quentropy(w[eta],1)
    for n in range(low, high+1):
        true_1n=howmany_top(eta,n,1/n, ifprint=False)[0]
        eps1n=low_eps(eta, n, 1/n)
        UB_1n=n*(entr+eps1n)
        
        true_12=howmany_top(eta,n,1/2, ifprint=False)[0]
        eps12=low_eps(eta, n, 1/2)
        UB_12=n*(entr+eps12)
        
        true_34=howmany_top(eta,n,3/4, ifprint=False)[0]
        eps34=low_eps(eta, n, 3/4)
        UB_34=n*(entr+eps34)
        print(n,n*entr,true_1n,UB_1n,true_12,UB_12,true_34,UB_34)

def tablepoints_count_til_entropy(eta,low,high, step=1):
    global w
    print("n p p_with_overhead")
    for n in range(low, high+1, step):
        print(n,count_til_entropy(eta,n,remove_overhead=True),count_til_entropy(eta,n,remove_overhead=False))

def tablepoints_count_til_entropy_fake_overhead(eta,low,high, step=1):
    global w
    print("n p_with_overhead p_with_fake_overhead p")
    for n in range(low, high+1, step):
        d=count_til_entropy(eta,n,fake_overhead=True)
        e=count_til_entropy(eta,n,remove_overhead=False)
        print(n,d[0],d[1],e)

In [3]:
# Old method, might be up for deletion

class parlists:
    def __init__(self, const, dic, num, p):
        self.c = const
        self.dic = dic
        self.len = len(dic)
        self.count = num
        self.p = p
    
    def __repr__(self):
        if self.c<0:
            return("List with " + str(self.len) + " clusters, with probability at least 2^(-H(.)n+" + str(-self.c) + "), total probability is " + str(float(self.p)))
        else:
            return("List with " + str(self.len) + " clusters, with probability at least 2^(-H(.)n-" + str(self.c) + "), total probability is " + str(float(self.p)))
        
empty_list=parlists(-float('inf'),[],0,0)

J=[]
global_num=0
global_Lp=0
global_p=[]
global_m=[]
global_log_p=[]

class distributions:
    def __init__(self, dist):
        cumu=0
        for key in dist:
            cumu+=dist[key]
        if cumu!=1:
            print("Not a probability distribution")
        else:
            probabilities, multiplicities=multiplicity(dist)
            eta=len(probabilities)-1
            self.eta = eta
            self.d = dist
            self.dist = dist
            self.p = probabilities
            self.log_p = []
            for prob in self.p:
                self.log_p += [-truelog(prob,2)]
            self.m = multiplicities
            self.entropy = dist_entropy(dist, 1)
            self.kcl={}
            self.keys=[]
    
    def prob(self, i):
        return self.d.get(i,0)
    
    def vec_prob(self, L):
        p=0
        if len(L)-1<=self.eta:
            p=1
            for i in range(len(L)):
                p*=self.prob(i)**L[i]
        return p
    
    def kc(self, n):
        return self.kcl.get(n,empty_list)
    
    def sam_parlist(self, n, c=0, if_ret=True, if_print=True):
        global J, global_p, global_m, global_num, global_log_p
        if self.kc(n).c<c:
            if self.eta==0:
                self.kcl[n]=parlists(c,[[n]])
            else:
                J=[]
                global_p=self.p
                global_m=self.m
                global_num=0
                global_Lp=0
                global_log_p=self.log_p
                gc.collect()
                create_parlist(self.eta,self.entropy*n+c,0,n,(self.eta+1)*['-'],1, if_print=if_print)
                J.sort(key=lambda x: x[1], reverse=True)
                self.kcl[n]=parlists(c,J.copy(), global_num, global_Lp)
                self.keys+=[n]
                global_num=0
                J=[]
                global_p=[]
                global_m=[]
                global_log_p=[]
                global_num=0
                global_Lp=0
                gc.collect()
                out=self.kc(n)
        elif self.kc(n).c>c:
            out=[]
            for key in kc(n).dic:
                if truelog(kc(n).dic[key][1])<-self.entropy*n-c:
                    break
                out+=[kc(n).dic[key]]
        else:
            out=self.kc(n)
        if if_ret:
            return self.kc(n)
        
    def count_til_p(self,n,c=0,ret_l=True, if_print=True):
        if self.kc(n).c<c:
            self.sam_parlist(n,c,if_print=if_print)
            out=self.kc(n).count
        elif self.kc(n).c==c:
            out=self.kc(n).count
        else:
            out=0
            i=0
            L=self.kc(n).dic
            while i<len(L) and truelog(L[i][1])>-self.entropy*n-c:
                out+=L[i][2]
                i+=1
        if ret_l:
            return truelog(out)
        return out
    
    def make_tablepoints(self, low_n, high_n, c=0, delete_after=False):
        print("n p coresize Eclassic Equantum")
        for n in range(low_n, high_n+1):
            L=self.sam_parlist(n, c=c, if_ret=True, if_print=False)
            p=0
            count=0
            ELZ=0
            Equantum=0
            for l in L.dic:
                p+=l[1]*l[2]
                ELZ+=l[1]*(count+l[2]*(l[2]+1)/2)
                Equantum+=l[1]*approx_sum_of_roots(count+1,count+l[2])
                count+=l[2]
            ELZ+=(1-p)*count
            print(n,format(float(p),'f'),truelog(count),truelog(ELZ),truelog(Equantum))
            if delete_after:
                del(self.kcl[n])
                gc.collect()
            
            
            
            
    
        
def count_L(L,m):
    out=1
    for i in range(len(L)):
        out*=m[i]**L[i]
    return out*multinomial(L)

def create_parlist(eta,p_limit,cur_j,high_n,L,p, if_print=True):
    global J, global_p, global_Lp, global_m, global_num, global_log_p
    if cur_j==eta and global_log_p[eta]*high_n<=p_limit:
        L[cur_j]=high_n
        if if_print:
            clear_output(wait=True)
            print(L)
        count=count_L(L, global_m)
        J+=[[L.copy(),p*global_p[cur_j]**high_n,count]]
        global_num+=count
        L[cur_j]='-'
    elif high_n==0:
        for j in range(cur_j, eta+1):
            L[j]=0
        if if_print:
            clear_output(wait=True)
            print(L)
        count=count_L(L, global_m)
        J+=[[L.copy(),p,count]]
        global_num+=count
        for j in range(cur_j, eta+1):
            L[j]='-'
    elif cur_j<eta:
        if p_limit//global_log_p[cur_j]>=high_n:
#            print(global_log_p[cur_j])
#            print(floor(p_limit/global_log_p[cur_j])+1)
            for i in range(high_n+1):
#                if cur_j==0:
#                    print(i)
                L[cur_j]=i
#                print(L)
                create_parlist(eta,p_limit-i*global_log_p[cur_j],cur_j+1,high_n-i,L,p*global_p[cur_j]**i, if_print=if_print)
#                L[cur_j]='-'

CBD_list={0:distributions({0:1}),1:distributions({-1:1/4,0:1/2,1:1/4})}

def CBD(eta):
    global CBD_list
    CBD_list[eta]=CBD_list.get(eta,distributions(self_convolution(CBD_list[1].d,eta)))
    return CBD_list[eta]


falcon512dist={-20: 1/1048576,
 -19: 1/524288,
 -18: 3/1048576,
 -17: 19/1048576,
 -16: 43/1048576,
 -15: 117/1048576,
 -14: 65/262144,
 -13: 19/32768,
 -12: 1323/1048576,
 -11: 2613/1048576,
 -10: 4887/1048576,
 -9: 4359/524288,
 -8: 7321/524288,
 -7: 23297/1048576,
 -6: 34803/1048576,
 -5: 187/4096,
 -4: 63141/1048576,
 -3: 609/8192,
 -2: 45531/524288,
 -1: 25219/262144,
 0: 52049/524288,
 1: 25219/262144,
 2: 45531/524288,
 3: 609/8192,
 4: 63141/1048576,
 5: 187/4096,
 6: 34803/1048576,
 7: 23297/1048576,
 8: 7321/524288,
 9: 4359/524288,
 10: 4887/1048576,
 11: 2613/1048576,
 12: 1323/1048576,
 13: 19/32768,
 14: 65/262144,
 15: 117/1048576,
 16: 43/1048576,
 17: 19/1048576,
 18: 3/1048576,
 19: 1/524288,
 20: 1/1048576}

falcon1024dist={-13: 3/1048576,
 -12: 11/524288,
 -11: 109/1048576,
 -10: 307/1048576,
 -9: 1055/1048576,
 -8: 1465/524288,
 -7: 3711/524288,
 -6: 8099/524288,
 -5: 3983/131072,
 -4: 55367/1048576,
 -3: 84393/1048576,
 -2: 113981/1048576,
 -1: 2151/16384,
 0: 72973/524288,
 1: 2151/16384,
 2: 113981/1048576,
 3: 84393/1048576,
 4: 55367/1048576,
 5: 3983/131072,
 6: 8099/524288,
 7: 3711/524288,
 8: 1465/524288,
 9: 1055/1048576,
 10: 307/1048576,
 11: 109/1048576,
 12: 11/524288,
 13: 3/1048576}